# Homework 3: Word Embeddings
In this homework, we will try to approximate a Skip-gram word embedding via positive pointwise mutual information (PPMI) and truncated singular value decomposition (SVD). 

## The setup
Let's import the required libraries and load the data for preparing our word vectors. We are going to load a list of movie plot summaries (http://www.cs.cmu.edu/~ark/personas/) and use that as our corpus. You do not need to modify them.

In [0]:
# This code gets the data file from github and imports them into Colab
%%capture
!wget https://raw.githubusercontent.com/dbamman/nlp20/master/HW_3/plot_summaries_tokenized.csv

In [0]:
import numpy as np
import pandas as pd
from string import punctuation
from collections import Counter, defaultdict
from math import log2
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import svds

def load_data():
    """
    Loads the data and returns tokenized summaries.
    
    :return summaries_tokenized: a list that contains tokenized summaries text
    """
    df = pd.read_csv("plot_summaries_tokenized.csv")
    summaries_tokenized = list(df['SUMMARY'].apply(lambda text: text.split()))
    return summaries_tokenized

summaries = load_data()
num_summaries = len(summaries)
print("There are {} summaries.".format(num_summaries))
print("Example tokenized summary:", summaries[0])

There are 42303 summaries.
Example tokenized summary: ['Shlykov', 'a', 'hardworking', 'taxi', 'driver', 'and', 'Lyosha', 'a', 'saxophonist', 'develop', 'a', 'bizarre', 'lovehate', 'relationship', 'and', 'despite', 'their', 'prejudices', 'realize', 'they', 'arent', 'so', 'different', 'after', 'all']


We have ~42000 summaries containing ~13000000 words. We will now proceed by creating a vocabulary and will limit its size to something computationally feasible. You may find python's collections.Counter function useful. You may not import any additional libraries.

# 1. Create Vocabulary
We will start from creating our vocabulary. Vocabulary contains unigrams and their counts.

In [0]:
###################
# Do not modify
###################
min_count = (1 / 100) * len(summaries)
max_count = (1 / 10) * len(summaries)

In [0]:
summaries = load_data()
vocab = Counter(summaries[1])
{x : vocab[x] for x in vocab if vocab[x] >= 1}
{x : vocab[x] for x in vocab if vocab[x] <= 5}
word2idx, idx2word = {}, {}
##################################
new_list = list(vocab.keys())
for i in range(len(new_list)):
    word2idx[new_list[i]] = i
    idx2word[i] = new_list[i]

print(word2idx)

{'The': 0, 'nation': 1, 'of': 2, 'Panem': 3, 'consists': 4, 'a': 5, 'wealthy': 6, 'Capitol': 7, 'and': 8, 'twelve': 9, 'poorer': 10, 'districts': 11, 'As': 12, 'punishment': 13, 'for': 14, 'past': 15, 'rebellion': 16, 'each': 17, 'district': 18, 'must': 19, 'provide': 20, 'boy': 21, 'girl': 22, 'between': 23, 'the': 24, 'ages': 25, '12': 26, '18': 27, 'selected': 28, 'by': 29, 'lottery': 30, 'annual': 31, 'Hunger': 32, 'Games': 33, 'tributes': 34, 'fight': 35, 'to': 36, 'death': 37, 'in': 38, 'an': 39, 'arena': 40, 'sole': 41, 'survivor': 42, 'is': 43, 'rewarded': 44, 'with': 45, 'fame': 46, 'wealth': 47, 'In': 48, 'her': 49, 'first': 50, 'Reaping': 51, '12yearold': 52, 'Primrose': 53, 'Everdeen': 54, 'chosen': 55, 'from': 56, 'District': 57, 'Her': 58, 'older': 59, 'sister': 60, 'Katniss': 61, 'volunteers': 62, 'take': 63, 'place': 64, 'Peeta': 65, 'Mellark': 66, 'bakers': 67, 'son': 68, 'who': 69, 'once': 70, 'gave': 71, 'bread': 72, 'when': 73, 'she': 74, 'was': 75, 'starving': 76, 

In [0]:
def creat_vocabulary(tokenized_documents, min_count, max_count):
    """
    This function takes in tokenized documents and returns a
    vocabulary and word <-> index lookup dictionary of some frequently appearing words.
    
    :param tokenized_documents: a list of tokenized strings
    :param min_count: minimum unigram count
    :param max_count: maximum unigram count
    :return vocab: a Counter where vocab[word] = count of word's occurences in all the documents
    :return word2idx: a word -> index lookup Dictionary for words in vocab.
    :return idx2word: a index -> word lookup Dictionary for words in vocab.
    """
    # 1a. Compute unigram counts. A unigram is a single word, e.g. foo
    vocab = Counter()
    ##################################
    #for word in tokenized_documents:
    #  if word not in vocab.keys():
    #    vocab[word] = 1
    #  else:
    #    vocab[word] += 1
    #counts = Counter(tokenized_documents)
    #for xs in tokenized_documents:
    #  for x in set(xs):
    #    vocab[x] += 1
    vocab = Counter([lis for sublist in tokenized_documents for lis in sublist])
    ##################################
      
    # 1b. Remove unigrams that has #(unigram) < min_count or #(unigram) > max_count
    # to eliminate unigrams occurring very frequently or infrequently. 
    # This will limit its size to something computationally feasible.
    print('%d vocabs before' % len(vocab))
    ##################################
    for key, cnts in list(vocab.items()):   # list is important here
       if cnts < min_count or cnts > max_count:
          del vocab[key]
    #vocab = Counter(x for x in vocab if (vocab[x] >= min_count and vocab[x] <= max_count))
    #vocab = {x : vocab[x] for x in vocab if vocab[x] >= min_count and vocab[x] <= max_count}
    #vocab = {x : vocab[x] for x in vocab if vocab[x] < max_count}

    ##################################
    print('%d vocabs after' % len(vocab))
          
    # 1c. Build word <-> index lookup for words in vocab.
    word2idx, idx2word = {}, {}
    ##################################
    new_list = list(vocab.keys())
    for i in range(len(new_list)):
      word2idx[new_list[i]] = i
      idx2word[i] = new_list[i]

    ##################################
    
    return vocab, word2idx, idx2word

In [0]:
vocab, word2idx, idx2word = creat_vocabulary(summaries, min_count, max_count)

214147 vocabs before
2750 vocabs after


# 2. Build Term-Context Matrix

In [0]:
###################
# Do not modify
###################
window_size = 3

In [0]:
def build_term_context_matrix(tokenized_documents, vocab, window_size):
    """
    This function returns a `word_pair_count` Counter with each 
    word_pair_count[(w, c)] = number of times the word `c` occurs in the context of word `w`. (where `w`, `c` belong to the vocab)
    To make it efficient, instead of building the sparse term-context matrix, 
    we will build 3 separate Counters: word_pair_count, w_count, c_count
    You may find python's Counter useful here

    :param tokenized_documents: a list of tokenized strings
    :param vocab: vocabulary Counter
    :param window_size: context window size 
    :return word_pair_count: a Counter where word_pair_count[(w, c)] = count of c's occurences in w's context window, i.e. #(w, c)
    :return w_count: a Counter where w_count[w] = the number of times w occured in the documents, i.e. #(w)
    :return c_count: a Counter where c_count[c] = the number of times c occured in the documents, i.e. #(c)
    """
    word_pair_count = Counter()  
    w_count = Counter()
    c_count = Counter()
    ##################################
    #(word, context word): number of times they appear together 
    #for each word in a document
      #generate all possible tuples between target word and context word around it within window (f in vocab)
        #for each tuple add to word_pair_count Counter 
          #
    for document in tokenized_documents: 
      for i, word in enumerate(document):
        if word in vocab.keys():
          for j in range(max(i-window_size, 0), min(i+window_size, len(document))):
             if document[j] in vocab.keys() and document[j] != word:
                word_pair_count[word, document[j]] += 1
    
    for word_pair in word_pair_count:
      w_count[word_pair[0]] += 1
      c_count[word_pair[1]] += 1
    
    print(len(w_count), len(c_count))
    ##################################
    
    return word_pair_count, w_count, c_count

In [0]:
len(vocab)

2750

In [0]:
word_pair_count, w_count, c_count = build_term_context_matrix(summaries, vocab, window_size)
print("There are {} word-context pairs".format(len(word_pair_count)))

# The number of w_count and c_count should match your number of vocab
assert len(w_count) == len(vocab)
assert len(c_count) == len(vocab)

2750 2750
There are 1606462 word-context pairs


# 3. Build Positive Pointwise Mutual Information (PPMI) Matrix
In this part, you will build a PPMI matrix using Scipy's Compressed Sparse Column matrix to save storage space. (https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csc_matrix.html)

Sparse matrix is a matrix which contains very few non-zero elements. When a sparse matrix is represented with a 2-dimensional array, we waste a lot of space to represent that matrix. In NLP application, it's quite common to use sparse matrix since the size of vocabulary is usually very large. 

Below is an example of how to build a sparse matrix where `data`, `row` and `col` satisfy the relationship `M[row[k], col[k]] = data[k]`.

```python
>>> row = np.array([0, 2, 2, 0, 1, 2])
>>> col = np.array([0, 0, 1, 2, 2, 2])
>>> data = np.array([1, 2, 3, 4, 5, 6])
>>> M = csc_matrix((data, (row, col)))
>>> M.toarray()
array([[1, 0, 4],
       [0, 0, 5],
       [2, 3, 6]])
```

Recall that
$$
\begin{gather*}
  \text{PMI}(w, c) = \log_2 \frac{P(w, c)}{P(w)P(c)} \\
  \text{PPMI}(w, c) = \max(0, \text{PMI}(w, c))
\end{gather*}
$$
You should use `log2` function from the math package that is alreadly imported for you.

In [0]:
def build_PPMI_matrix(word_pair_count, w_count, c_count, word2idx):
    """
    This function returns a PPMI matrix represented by a csc sparse matrix.

    :params word_pair_count: a Counter where word_pair_count[(w, c)] = count of c's occurences in w's context window
    :params w_count: a Counter where w_count[w] = the number of times w occured in the documents
    :params c_count: a Counter where c_count[c] = the number of times c occured in the documents
    :params word2idx: a word -> index lookup Dictionary for words in vocab
    :return PPMI: PPMI csc sparse matrix
    """
    data, rows, cols = [], [], []
    total_occurences = sum(word_pair_count.values())
    for (w, c), n in word_pair_count.items():
      ##################################
      rows.append(word2idx[w])
      cols.append(word2idx[c])
      pwc = n / total_occurences
      nw = w_count[w]
      pw = nw / total_occurences
      nc = c_count[c]
      pc = nc / total_occurences

      pmi = np.log2(pwc/(pw*pc))   
      ppmi = max(pmi, 0)

      data.append(ppmi)
      ##################################

    PPMI = csc_matrix((data, (rows, cols)))
    return PPMI

In [0]:
PPMI = build_PPMI_matrix(word_pair_count, w_count, c_count, word2idx)

# The shape of PPMI matrix should match your number of vocab
assert PPMI.shape == (len(vocab), len(vocab))

# 4. Truncated SVD
In this part, we will obtain a dense low-dimensional vectors via truncated (rank-k) SVD. You should use `svds` function from Sicpy that is already imported for you to obtain the SVD factorization.
(https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.svds.html)


In [0]:
###################
# Do not modify
###################
rank = 20

In [0]:
def get_embeddings(PPMI, rank):
    """
    Reutrns the left singular vectors as word embeddings via truncated SVD

    :params PPMI: PPMI csc sparse matrix
    :params rank: number of singular values and vectors to compute
    :return u: left sigular vectors from sprase SVD
    :return s: the singular values from sparse SVD
    """
    ##################################
    u, s, v = svds(PPMI, rank) 
    ##################################

    return u, s

In [0]:
embeddings, _ = get_embeddings(PPMI, rank)
embeddings /= np.linalg.norm(embeddings, axis=1, keepdims=True)  # Normalize embeddings matrix

# The shape of the embeddings matrix should be (# vocab, rank)
assert embeddings.shape == (len(vocab), rank)

# Make sure embeddings is normalized
assert True == np.isclose(np.linalg.norm(embeddings[0]), 1)

# 5. Evaluate Word Embeddings via Cosine Similarity

Using cosine similarity as a measure of distance [§6.4 Jurafsky & Martin](https://web.stanford.edu/~jurafsky/slp3/6.pdf), we will now find the closest words to a certain word. We define cosine similarity as, $$cosine(\overrightarrow{v},\overrightarrow{w}) = \frac{\overrightarrow{v} \cdot \overrightarrow{w}}{\vert v \vert \vert w \vert}$$

Please complete the function below that calculates the 'K' closest words from the vocabulary. You may not use any additional libraries.

In [0]:
###################
# Do not modify
###################
num_neighbors = 5

In [0]:
def cosine_distances(matrix, vector):
    """
    The function takes in a matrix and a vector (both normalized) 
    and returns the cosine distances for this vector against all others.
    The pretrained embeddings are normalized.

    :params matrix: word embeddings matrix
    :params vector: word vector for a particular word
    :return distances: a cosine distances vector
    """
    ##################################
    #distances = []
    ##sqrt_v = np.sqrt(sum(map(lambda i : i * i, vector)))
    #row_sum = 0
    #sqrt_m = 0
    #for row in range(0,len(matrix)):
    #  row_sum = 0 
    #  sqrt_m = 0
    #  for col in range(0, len(matrix[0])):
    #    row_sum += vector[col]*col
    #    sqrt_m += col**2
   # 
   # cos_sim = row_sum/(np.sqrt(sqrt_m)*sqrt_v)
   # distances.append(cos_sim)
    #sqrt_v = np.sqrt(sum(map(lambda i : i * i, vector)))
    #remove distances that equals 1
    distances = []

    for row in matrix:
      #if all(vector == row):
      #  continue
      dot_prod = np.dot(row, vector)
      #sqrt_m = np.sqrt(sum(map(lambda i : i * i, row)))
      distances += [dot_prod]
      #distances.append(dot_prod/(np.sqrt(np.dot(vector, vector))*np.sqrt(np.dot(row, row))))
    ##################################
    
    return  distances


def nearest_neighbors(embeddings, word, k, word2idx, idx2word):
    """
    For each query word, this function returns the k closest words from the vocabulary.

    :params embeddings: word embedding matrix
    :params word: query word
    :params k: number of cloest words to return
    :params word2idx: a word -> index lookup dictionary
    :params idx2word: a index -> word lookup dictionary
    :return nearest_neighbors: a list of cloest words
    """
    vector = embeddings[word2idx[word]]
    distances = cosine_distances(embeddings, vector)
    nearest_neighbors = []
    ##################################
    distances = [-x for x in distances]
    sidxs = np.argsort(distances)  
    nearest_neighbors = [(idx2word[sidx]) for sidx in sidxs[1:k+1]]

    #nearest_neighbors = [(idx2word[sidx]) for sidx in sidxs[-k-1:]]
    print(distances[1:k+1])
    ##################################
    
    return nearest_neighbors

In [0]:
query_words = ["doctor", "zombie", "robot", "eat", "bus"]
for word in query_words:
    print(word, nearest_neighbors(embeddings, word, num_neighbors, word2idx, idx2word))

[-0.09530300920532074, 0.14579087330724647, -0.1791963917384067, 0.17279026182136192, 0.15324360592958883]
doctor ['priest', 'Christine', 'Elizabeth', 'Anna', 'teacher']
[-0.11659300051966515, 0.04014627682205825, 0.1743977823611117, -0.04639244722835259, 0.013102248835768937]
zombie ['infected', 'demon', 'possessed', 'zombies', 'creatures']
[-0.1010706002095609, 0.009389637154399093, 0.1524761674263763, 0.1463347976236481, 0.2544234625962456]
robot ['alien', 'creature', 'demon', 'monster', 'weapon']
[-0.09820479843633131, -0.21388530195443306, 0.13548540876308418, -0.46292322911493017, 0.0138998977295548]
eat ['throw', 'wear', 'sleep', 'walk', 'look']
[-0.3851950740067549, 0.2243232679041557, 0.04234978684747237, 0.04092279703186516, -0.09545347190707877]
bus ['road', 'truck', 'river', 'bridge', 'van']


# 6. Evaluate Word Embeddings via Analogous Tasks

The embedding space is known to capture the semantic context of words. An example of it is $\overrightarrow{woman} - \overrightarrow{man} \simeq \overrightarrow{queen} - \overrightarrow{king}$. Use the `cosine_distances()` function you wrote above to find such relations.

In [0]:
def relation(embeddings, query_words, word2idx, idx2word):
    """
    Takes in 3 words and returns the closest word (in terms of cosine similarity)
    to the normalized algebraic addition of the three vectors.
    The parameters follow this order : word_vec1 - word_vec2 ~ closest - word_vec3

    :params embeddings: word embedding matrix
    :params query_words: a list of query words in the following order: [word1, word2, word3]
    :params word2idx: a word -> index lookup dictionary
    :params idx2word: a index -> word lookup dictionary
    :return closet_word: the closest word for the relation
    """
    word1, word2, word3 = query_words
    if all(word in vocab for word in query_words):
      ##################################
      word1vec = embeddings[word2idx[word1]]
      word2vec = embeddings[word2idx[word2]]
      word3vec = embeddings[word2idx[word3]]
      word_summation = word1vec - word2vec + word3vec
      word_summation /= np.linalg.norm(word_summation, keepdims=True)
      distances = cosine_distances(embeddings, word_summation)
      distances = [-x for x in distances]
      sidxs = np.argsort(distances)
      closet_word = (idx2word.get(sidxs[1])) 



      
      ##################################
      
      return closet_word
    else:
      missing = [w for w in query_words if w not in vocab]
      raise Exception("missing {} from vocabulary".format(", ".join(missing)))

In [0]:
queries = [["doctor", "nurse", "king"], ["robot", "weapon", "bus"], ["sing", "song", "justice"], ["elderly", "kids", "teenager"], ["soldier", "wound", "telephone"]]
for query in queries:
  closet_word = relation(embeddings, query, word2idx, idx2word)
  print("{} - {} ~= {} - {}".format(query[0], query[1], closet_word, query[2]))

doctor - nurse ~= Emperor - king
robot - weapon ~= river - bus
sing - song ~= protect - justice
elderly - kids ~= journalist - teenager
soldier - wound ~= retired - telephone
